<a href="https://colab.research.google.com/github/ruthvik-17/faq_search/blob/master/T5_Question_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown -O  t5_que_gen.zip --id 1vhsDOW9wUUO83IQasTPlkxb82yxmMH-V
!unzip t5_que_gen.zip

Downloading...
From: https://drive.google.com/uc?id=1vhsDOW9wUUO83IQasTPlkxb82yxmMH-V
To: /content/t5_que_gen.zip
1.65GB [00:15, 106MB/s] 
Archive:  t5_que_gen.zip
   creating: t5_que_gen_model/
   creating: t5_que_gen_model/t5_base_tok_que_gen/
  inflating: t5_que_gen_model/t5_base_tok_que_gen/spiece.model  
 extracting: t5_que_gen_model/t5_base_tok_que_gen/added_tokens.json  
 extracting: t5_que_gen_model/t5_base_tok_que_gen/tokenizer_config.json  
  inflating: t5_que_gen_model/t5_base_tok_que_gen/special_tokens_map.json  
   creating: t5_que_gen_model/t5_base_que_gen/
  inflating: t5_que_gen_model/t5_base_que_gen/config.json  
  inflating: t5_que_gen_model/t5_base_que_gen/pytorch_model.bin  
 extracting: t5_que_gen_model/logs.zip  
   creating: t5_ans_gen_model/
   creating: t5_ans_gen_model/t5_base_tok_ans_gen/
  inflating: t5_ans_gen_model/t5_base_tok_ans_gen/spiece.model  
  inflating: t5_ans_gen_model/t5_base_tok_ans_gen/added_tokens.json  
 extracting: t5_ans_gen_model/t5_base_

In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 7.2MB/s 
     |████████████████████████████████| 3.0MB 36.2MB/s 
     |████████████████████████████████| 1.1MB 33.8MB/s 
     |████████████████████████████████| 890kB 43.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=28fb826bc23ef9835353b6a41f00dfb54a0c472122b5e64bdb3cb8b90f6ce1de
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import argparse
import glob
import os
import json
import time
import logging
import random
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
class QueGenerator():
  def __init__(self):
    self.que_model = T5ForConditionalGeneration.from_pretrained('./t5_que_gen_model/t5_base_que_gen/')
    self.ans_model = T5ForConditionalGeneration.from_pretrained('./t5_ans_gen_model/t5_base_ans_gen/')

    self.que_tokenizer = T5Tokenizer.from_pretrained('./t5_que_gen_model/t5_base_tok_que_gen/')
    self.ans_tokenizer = T5Tokenizer.from_pretrained('./t5_ans_gen_model/t5_base_tok_ans_gen/')
    
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    self.que_model = self.que_model.to(self.device)
    self.ans_model = self.ans_model.to(self.device)
  
  def generate(self, text):
    answers = self._get_answers(text)
    questions = self._get_questions(text, answers)
    output = [{'answer': ans, 'question': que} for ans, que in zip(answers, questions)]
    return output
  
  def _get_answers(self, text):
    # split into sentences
    sents = sent_tokenize(text)

    examples = []
    for i in range(len(sents)):
      input_ = ""
      for j, sent in enumerate(sents):
        if i == j:
            sent = "[HL] %s [HL]" % sent
        input_ = "%s %s" % (input_, sent)
        input_ = input_.strip()
      input_ = input_ + " </s>"
      examples.append(input_)
    
    batch = self.ans_tokenizer.batch_encode_plus(examples, max_length=512, pad_to_max_length=True, return_tensors="pt")
    with torch.no_grad():
      outs = self.ans_model.generate(input_ids=batch['input_ids'].to(self.device), 
                                attention_mask=batch['attention_mask'].to(self.device), 
                                max_length=32,
                                # do_sample=False,
                                # num_beams = 4,
                                )
    dec = [self.ans_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
    answers = [item.split('[SEP]') for item in dec]
    answers = chain(*answers)
    answers = [ans.strip() for ans in answers if ans != ' ']
    return answers
  
  def _get_questions(self, text, answers):
    examples = []
    for ans in answers:
      input_text = "%s [SEP] %s </s>" % (ans, text)
      examples.append(input_text)
    
    batch = self.que_tokenizer.batch_encode_plus(examples, max_length=512, pad_to_max_length=True, return_tensors="pt")
    with torch.no_grad():
      outs = self.que_model.generate(input_ids=batch['input_ids'].to(self.device), 
                                attention_mask=batch['attention_mask'].to(self.device), 
                                max_length=16,
                                num_beams = 4)
    dec = [self.que_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
    return dec

In [5]:
que_generator = QueGenerator()

In [6]:
questions = ['Sure. I can help you with that!.  ➡ Your order will be Delivered in 14-28 days from the date of order & this applies for both regular & custom sizes',
 'Let\'s run through the steps ✔️. . 2. On the page, you can either choose from the mentioned sizes or look for the option "Please click here to order a custom size". 3. Enter the <i>desired length and width</i> in the appropriate columns.  It\'s that simple!',
 '💸 Sure! Our prices start at Rs. 9,989 and vary based on size. You can get more information by checking the Shop page on the website. ',
 'Sure! We accept payments via . ✔️ Credit card. ✔️ Debit card . ✔️ EMI via credit card. ✔️ Cardless EMI via Zest Money . ✔️ Cash On Delivery (only to certain pin codes)',
 '➡️ Unfortunately, we are unable to modify the mattress size once an order is confirmed🙏. ➡️ However you can send an email to support@flomattress.com with your order number, requesting for cancellation. ➡️ Once you receive confirmation that your original order has been cancelled, please go ahead and <em>place a fresh order for the correct size</em> ✔️',
 'Here you go! .  1️⃣ Please send an email to support@flomattress.com  with your order number, requesting for cancellation. 2️⃣ We will confirm within 48 hours that your order has been cancelled and payment has been refunded (if applicable). 3️⃣ If for any reason you don’t hear back from us within 48 hours, please call us on +91 73878 87750',
 'A great choice for those who prefer softer ergonomic support ✔️.',
 "Great! Let's check that out ✔️.",
 "Flo is India's most intelligently designed mattress in a box with 2 variants✔️.",
 'Sure! The Flo mattress now comes in 6 & 8 inches thickness ✔️.  Please refer to the chart below for available sizes 👇.',
 'We have designed our mattresses with features which you will care for the most ✔️. Which variant would you like to explore?.',
 '➡️ The zippered cover that comes with the mattress is not waterproof. ➡️ We highly recommend a waterproof mattress protector as it will help enhance the life of your mattress. ➡️ We recommend the Ultra Absorbent Mattress Protector from <mark>Hush</mark> and you can reach out to the Hush team on +91 77188 11177. ➡️ Don’t forget to ask for the 20% discount 🏃',
 'Great! Please select from one of the following categories 👇',
 'Absolutely!.  ✌️ <u>On a scale of 1-10</u>. 1 being as soft as a <em>cotton ball </em> . 10 being as hard as a <em>concrete floor</em>. ✔️ The top surface of the Ergo is a . 6/10.  ✔️ Whereas, that of Ortho is a 7/10. ✔️ Also, the bottom surface of both variants is an 8/10',
 '👍 Absolutely! <em>We offer a 3 month 0% EMI payment option via Zest Money</em>. 👍 <em>Please call us on +91 73878 87750 if you’d like to learn more about EMI</em>',
 'Sure! Please select from the options below 👇',
 'The Ortho is the firmer variant and recommended for those who are over 60 or those who have back issues ✔️.',
 'Sorry! The thickness of the mattress cannot be customised and we only make our mattresses in 6  and 8 inches thickness',
 '✔️ Say hello to risk-free mattress shopping. ✔️ We have a 100-night trial offer & no questions asked 😲. ✔️ If the mattress is not the right fit, you can exchange or refund it. ✔️ This offer applies to all sizes and across all channels!. ',
 "✅ Yes! We offer a 10-year warranty ✌️ on our mattresses which <em>covers manufacturing defects</em>. ✅ Although, we don't cover normal wear and tear or damages caused due to abuse. ✅ In case of a defect, please reach out to us on support@flomattress.com and don’t forget to mention your order number. ",
 'You are at the right place! Please check the options below 👇',
 "Sure! Here's what I can help you with 👇",
 'Absolutely!.  ➡ 99% of the pin codes in India are covered through our logistics partner FedEx 📦. ➡ If for any reason your pin code cannot be serviced by FedEx, your order will be cancelled and the entire payment will be refunded within 48 hours of order confirmation 🙏',
 '📞 Please call our support team on +91 73878 87750 and <em>we will try our best to find a way to get your order delivered to you as quickly as possible</em>',
 'Sorry! We don’t sell  spring mattresses. ➡️ Spring mattress - Fewer points of contact between your body and your mattress when you sleep. ➡️ Foam mattress - Millions of touch points between your body and your mattress when you sleep. ➡️As a result, foam mattresses tend to be a lot more <em>responsive to your body’s weight and shape & offer far greater support</em>',
 'Not to worry! .  We offer <mark>free home delivery</mark> to all of the pin codes that FedEx services 📦. If in case FedEx cannot deliver to your pin code, <em>we will reach out to you within 48-72 hours to either request an alternative delivery address and pin code or cancel your order</em>',
 '➡️ Sure! You can get 5% discount on your orders by clicking the link on the top of our home page 😊.',
 'Here you go! 😊. <u>Ergo:</u>. ➡️ The density of the top surface is 65 kg/m3 and that of the bottom surface is 40 kg/m3. <u>Ortho:</u>. ➡️ The density of the top surfacer is 28 kg/m3 and that of the bottom surface is 40 kg/m3',
 "It's very simple!. You will need to choose the online payment option and once you are on the payment gateway you will see all available EMI options.",
 'Not to worry!.  We absorb the cost of return shipping should you decide to return your Flo mattress 😊',
 '➡️ We do not have any distributors/retailers/showrooms ✖.  ➡️ We only sell online and that is how we are able to keep our prices so low 💰. ➡️ Feel free to 📞 us on +91 73878 87750 if you need help',
 '➡️ Unfortunately, we currently don’t offer any colour options. ➡️ We recommend buying bed sheets or mattress protectors in your desired colours instead!',
 '✅ We are working at 1/3rd capacity as per the latest government directive, and our courier partners also have reduced manpower due to COVID-19. ✅ While our best efforts are to get you your order as soon as possible, we humbly request you to be patient and understand the situation during these difficult times in case of delays.  . Thank you for your patronage and support. Stay Safe!. ',
 "Let's check what Flobot has to say on this 👇. ➡️ Low costs, more savings.  We only retail online cutting down the traditionally invest like showroom rent, electricity, etc ✔️. ➡️ Reengineered supply chain.  There are no middlemen hence bring you over 50% in savings ✔️. ",
 'Sure! Here you go 👇.',
 "Sure! Let's check them out ✔️.",
 "Thanks for participating! All contest winners are notified around 5 pm daily.  . We are sorry if you didn't win this time! Please stay tuned for future contests where we promise more exciting prizes!",
 'We are a foam-based mattress and you can learn more about each variant by clicking on it below',
 'Our mattresses weigh between 13 and 35 kgs depending on size',
 'We regret the inconvenience caused. Please send an email at escalations@flomattress.com so our team can address your concerns on a priority basis',
 '✔️ We use the 3D spacer fabric on the periphery of the mattress to dissipate heat, and the Ergo also comes with gel-infused memory foam. ',
 'Our mattresses are reversible and can be used either side',
 'No, our mattress in non-foldable but reversible and can be used either side.  😊',
 'Yes! The Ergo variant has a top layer of memory foam',
 'We apologise for the inconvenience caused.  . We might not be reachable due to network issues or unexpectedly high call volumes. Kindly wait a while and contact us again',
 'sorry to hear that!. Could you kindly reply, in a single message, what went wrong?',
 'Easy!.  ➡ Your Mobile Number is verified by a 6-digit OTP sent by MyTeam11. ➡ Enter and the OTP to verify your Mobile Number',
 'It seems you are checking the joined league in the wrong section. If you join a league in Safe Play Mode then it will not show under Regular Play Mode and vice versa.  . Kindly, check both the playing Mode, just by tapping the button available at the top of the home page, and then check',
 'The Unutilized amount is the amount that you add to your MyTeam11 wallet.  It can only be used for joining contests on MyTeam11, not for withdrawals',
 'MyTeam11 involves transactions with real money, which is why users are required to provide an identity proof for the same.  PAN card verification is used for verifying your identity and tracking TDS deductions',
 'When there is a tie between winners of a multiple winner contests, the prize money for the position of the winners and the immediately next winning position (if any), and so forth complying with the number of tied winners, will be summed up and then distributed equally among the tied winners. All other prize money will remain as per the winner’s standing',
 'Matches on MyTeam11 are divided into two different playing modes to make the game better and exciting—.  1. Regular Play: The deadline of the contest closes well-before the beginning of the original match, i.e. 1 hour before for Cricket & Basketball, and 1.5 hours before for Football & Kabaddi. 2. Safe Play: The deadline of the matches closes at the exact time the real-match is set to begin. This mode allows you to edit, create teams or join contests even after the playing XI is announced. This mode does not have any 1-hour-prior deadline and allows you to join with multiple teams (up to 10 teams) in matches accepting multiple entries',
 'You are given points based on the real-time performance of the players you select in your team. The points are awarded in accordance with the point system of MyTeam11.',
 'If someone tries to convert cash bonus to winnings by illegal means the user will be blocked from using the services of MyTeam11 and all the amount present in his/her wallet will be reduced to 0, resetting the user account',
 'Below are the types of bonuses/categories:.  ➡ Cash bonus is offered under various promotional schemes and offers.  ➡ Signup bonus is given on registering with platform . ➡ Unutilized amount is the amount that you add to your wallet. ➡ Surprise bonus is an undefined amount you get as extra bonus with scratch cards.  ➡ Winnings is the amount that you earn by winning contests',
 'The unutilized amount present in your MyTeam11 wallet can only be used for joining contests on MyTeam11, not for withdrawals',
 'Once registered with MyTeam11, you cannot use the same details for registration of another account.  It might be considered as a case of FairPlay Violation',
 'This is simple!.  ➡ Visit the Profile section. ➡ Click on the Info tab. ➡ Select Change Password option. ➡ Enter your old password, new password and click on Update Password. ',
 'You can use the Referral bonus amount earned by referring your friends to pay 25% of the entry fee of a league having 100 or more members and ₹25 or more entry fee',
 "The users participating in a cash contest win real money on the basis of their rank as the winning prize.  The amount is credited to the Winnings section of the user's wallet and can be withdrawn to his/her bank account by raising a withdrawal request",
 'The points of your Captain and Vice-Captain are deducted in the same manner as they are awarded i.e. 2x points for the Captain and 1.5x points for the Vice-Captain',
 'Instant Withdrawal & Instant PayTM withdrawals are recently introduced features with which you can get your withdrawal amount credited to your bank account within 2-3 hours. ➡ As a part of Instant withdrawals, a user can withdraw a minimum of ₹ 200 and a maximum amount of ₹ 10,000 in a day.  💡: It is an exclusive feature introduced for certain MyTeam11 users only. ',
 "The Cash Bonus amount can be defined as an amount added by MyTeam11 to user's wallet as a part of various promotional schemes or offers. The Cash Bonus is also given when the user registers on the platform for the first time",
 "That's easy!.  ➡ A confirmation link will be sent to your registered Email address. ➡ Click on the provided confirmation link to verify your Email.",
 'Once your first team is created, you can create multiple teams for a match.  To create multiple teams, select a match and click on the Create New Team tab. ',
 'MyTeam11 has 2 types of contests-.  1. Private contests: Created by users & can be joined using a unique code.  2. Public contests: Created by MyTeam11 & can be joined by anyone',
 'If you win a cash contest, the winning amount is instantly credited to your wallet.  You can also check the final ranks under the result section',
 'This is easy!.  To verify your bank account details with MyTeam11 you are required to provide the following bank account details:. ➡ Name of the bank. ➡ Name of the branch. ➡ Name of the account holder. ➡ Account Number. ➡ IFSC code',
 'There might be one or more winners for a single contest of MyTeam11. To know the number of winners for a contest —. ✔️ Select the contest and click on Winners.  ✔️ You can then view a detailed list of winners and pre-defined winnings',
 'The user must withdraw their winnings within 365 days of credit.  If not withdrawn, the amount is forfeited to MyTeam11',
 "That's easy!.  Once you register on MyTeam11, a verification link is sent to your registered Email ID, the Mobile Number is verified by OTP.  You can verify PAN card and Bank Account manually under the Profile section of the app. ",
 'MyTeam11 organizes various free and paid contests.  The Cash contest on MyTeam11 requires the user to pay a pre-defined entry fee to be able to join the contest. Such contests have defined cash winnings for the highest scoring users',
 'The Cash bonus earned as part of various promotional schemes of offers can be used for joining leagues on MyTeam11.',
 'A TDS tax deduction of 31.20% is made on winnings amounting to be more than ₹10,000.  ➡ MyTeam11 deducts the tax amount directly from user winnings before crediting it to the user. ➡ The users are provided a TDS certificate for the same',
 'You are awarded points on the basis of the performance of your selected players.  If any of your selected player is not included in the game, then no points will be awarded to the player. ',
 'Here are some basic Game rules to help you play with MyTeam11 👇.  ➡ You should be more than 18 years of age. ➡ You should be a non-resident of Odisha, Telangana, Nagaland, Sikkim or Assam. ➡ You can create only 1 MyTeam11 account with registered Email ID. ➡ Teams can only be edited before the deadline of the contest',
 'The signup bonus is awarded when you register on MyTeam11.🏆💸.',
 "The withdrawal amount (except for Instant Withdrawal) is credited to the user's bank account within 3-4 working days.  . The minimum amount to raise a withdrawal request is ₹ 200",
 "Contest that is not marked with C icon is known as non-confirmed leagues.  . When such leagues don't fill up their required number of participants at the time of deadline closing, they get cancelled and the entry fee is refunded to the users",
 "It's easy to add money to your wallet!👇.  ➡ Go to Profile section. ➡ Tap on Wallet. ➡ Click on Add Cash and add money to your MyTeam11 account through various methods. ☑️ Debit Card. ☑️ Credit Card. ☑️ Net banking. ☑️ Various payment wallets.",
 'On joining a contest, the account balance from your wallet is deducted as per the below cycle 👇. Unutilized ➡ Cash Bonus ➡  Winnings',
 "Thank you. I hope I've been helpful.  If you have a minute, please tap below to tell me how I did. I'd love an honest review 👇. ",
 "Hello. I'm here to assist you with all your queries. Please select the issue type from the list below and one of our agents will be able to assist you!.",
 'Just update your account with correct details and MyTeam11 will look into it',
 'Gameplay with MyTeam11 is very easy!.  ➡ Register and verify your account. ➡ Pick the sport and match you want to join. ➡ Create a virtual team. ➡ Select your captain and vice-captain and JOIN a contest.',
 "It's easy!.  To join a contest on MyTeam11—. ➡ Select a sport and a match to join. ➡ Pick the contest you wish to join from the available list. ➡ Click on the Join button in front of the specific contest",
 'The winner of a contest is decided on the basis of the ranks.  The team you create scores points on the basis of the real-time performance of your chosen players and the team with the highest aggregate points scores the highest rank',
 'The multiple winner contests have a pre-defined winning amount for every rank.  The prize is then distributed on the basis of the ranks of a user in accordance with the defined winnings list. Valid tax deductions are made on winnings of more than ₹10,000',
 'Points are not awarded to the users for a super over in a match, as the points scored or wickets taken by the players in a super over are not considered in their official records',
 'Choosing a Captain and Vice-Captain for your team will earn you extra points.  You earn points as 2x for your selected Captain and as 1.5x for your Vice-Captain',
 "You can add as many credit/debit cards as you wish.  Just make sure thay haven't expired!",
 'The users cannot claim a refund for an amount once the money is added to MyTeam11',
 'To make withdrawals from MyTeam11 your bank account details need to be verified with us.  Follow the below steps for Withdrawal:. ➡ Login to your MyTeam11 account. ➡ Click on the Profile section; the Wallet section will open up. ➡ Click on Withdraw option under Your Winnings section in the Wallet. ➡ Enter the Withdrawal amount  (minimum ₹200 and a maximum of ₹1 Lakh per day) and proceed by clicking Withdraw Now button',
 "👉 In case your Instant Withdrawal fails, we'll try processing it again within 10 minutes. 👉 If the second attempt fails, then the withdrawal will be processed as Normal Withdrawal and the amount will be credited within 4-5 working days.                                                       . ",
 'Sorry to hear that you are facing an issue logging in.  Select an option from below so I can help you resolve the issue 👇',
 "Here's how you can verify your Bank account — .  ➡  Visit the Verify tab under the Profile section of the MyTeam11 app. ➡ Select Verify in front of Bank Account and enter the details required. ➡ Click on Verify Now to proceed.  Bank account verification on MyTeam11 takes around 24 working hours",
 'You need to Verify your MyTeam11 account to be eligible for the following--. 1. Earning Signup/Refer Bonus. 2. Withdraw Winnings from your MyTeam11 account',
 'Creating a team on MyTeam11 is pretty simple!.  ➡ Select the match you want to play. ➡ Click on the Create Team tab. ➡ Select the required number of players in your virtual team within a defined budget. (Make sure you select your captain and vice-captain.).',
 'Steps to verify the PAN Card—.  ➡ Visit the Verify tab under the Profile section of MyTeam11. ➡ Select Verify in front of PAN card and enter the details required. ➡ Click on Verify Now to proceed.  PAN card verification on MyTeam11 takes around 24 working hours',
 'Sorry! We do not award any points for any player announced as Man of the Match',
 'Oops! Account verification can fail to missing or incorrect information.  Not to worry, in this case, an email is sent to your registered email ID with a failure reason and resolution suggestions',
 "No user is allowed to change his Team Name.  However, at the time of registration if any user didn't mention his Team Name then it will fetch the Team name from the email address and as an exception we allow the user to change his Team name. Such a user will have to send an email on support@myteam11.com with 3 unique Team names",
 'You can check your transactions only for the last 15 days.  Any issues related to transactions shall be reported within 7 days of the occurrence. 📑',
 'You can add a Credit or Debit card to your MyTeam11 account in these simple steps 👇.  ➡ Click on the Add Cash tab. ➡ Enter the amount you wish to add & select your payment gateway. ➡ Select a card and enter your card details. ➡ Click on Save and Proceed',
 'The minimum amount that you can add to your MyTeam11 wallet is ₹10 and there is no limit for the maximum amount.  Tap below to know more about cash refund.👇',
 'The sign-up bonus expires within 14 days from the day it is credited',
 'This is really easy!.  ➡ Click on the Profile Picture and go to the My Profile section. ➡ On the top right-hand side, you will get the Edit option where you can edit or change your Profile name',
 'The Referral Bonus earned under the Refer and Earn Policy is credited into the cash bonus section of your MyTeam11 wallet',
 "Don't worry! I can help you out.",
 "We're sorry you had to face that.",
 "Oh no! I can help. ✔️ First, ensure that you've entered the correct Email address.  ✔️ Do check your Inbox and Spam folder. ",
 'The winnings of a Cash Contest on MyTeam11 are distributed within a few hours of completion of the match.  The points are checked and cross-verified by our team experts and no users get compromised, then the winnings are distributed',
 'The cash bonus credited into your MyTeam11 wallet comes with a validity of 14 days, counted from the day of credit. ',
 'If you are found to be misusing your referral code to earn extra cash by creating a fake refer chain, then it will be considered a case of FairPlay Violation and will attract the required consequences',
 'Here are the steps to edit your team.  ➡ Select My Contest. ➡ Click on the contest for which you want to edit team. ➡ Click on Edit option available on the league you wish to edit. ➡ Make changes and it is reflected instantly on MyTeam11.  Note: Teams can only be edited before the deadline of the contest',
 "We would like to inform you that there wasn't any issue with the teams from our end, you can crosscheck the same by checking the points of your team and your opponent's team. You can also confirm the same by downloading the leaderboard pdf.  We would also like to inform you that Myteam11 has a certified gaming algorithm that doesn't allow even the programmers to intervene in the gaming interface. Also, the employees of MyTeam11 are not allowed to participate in the contests on MyTeam11",
 'We cannot predict the match result and the players.  We recommend that you may use your gaming skills, do analysis and then choose the best players to play on MyTeam11',
 "Free entry leagues or 100% Cash Bonus league on MyTeam11 are provided according to the popularity level of the match.  In addition, whenever a free entry league or Cash Bonus league will be organized on MyTeam11, you'll be notified",
 'There are no on-going issues with the Amount Add procedure.  We request you to please try to change your Payment method & then try to add money.',
 'A user is allowed to have only one single account registered with MyTeam11.  Having more than one account on MyTeam11 is considered to be a case of FairPlay Violation',
 'Once the PAN Card is verified with us, it cannot be deleted',
 "We apologize and regret the inconvenience caused due to delay in point updating.  A team of expert and experienced professionals cross-check the total points with the MyTeam11 fantasy scorecard and the live scores before finally updating the points and awarding ranks to the user. You can cross-check your team's points under the LIVE or RESULT section.",
 'If you want to reset your MyTeam11 account then you need to talk to our Customer Support Executive.  . Please wait while we transfer your chat to the concerned department',
 'We are glad to announce the New Rules! For Team Creation ⭐. As per the new rules you can select 👇.  2-6 Batters. 2-6 Bowlers. 1-6 All-Rounders. 1-4 Wicket Keepers.  Where previously it was 👇.  3-5 Batters. 3-5 Bowlers. 1-3 All-Rounders. 1 Wicket Keeper. We hope you enjoy the new features and updates 🎉',
 'In case a substitute replaces any on-field player in any circumstance, then the substitute will not be awarded any points for his/her part on the field.  . However, in case a player is announced as a Concussion Substitute, then the player shall be awarded (2) points for making an appearance on the field along with the points awarded on the basis of his/her on-field performance',
 "In order to create a private contest, first choose a match and create your team. After you've created your team, click on Create a Contest.  . Enter the details like Contest Name, Winning Amount, Contest Size(participants), in the required fields, and click on Create Contest",
 'We would like to inform you that people who reside out of India are allowed to participate on MyTeam11 if they have an Indian bank account and PAN card',
 'There is no issue from our side and it seems very unusual, We request you to please check your internet connectivity and try again to verify and if you are getting any error then . please share the screenshot of the same via email on support@myteam11.com',
 'As per your query we request you to please send us email on marketting@myteam11.com with the detailed information',
 'We really appreciate your effort to make our services better. We would like to inform you that leagues on MyTeam11 are provided according to the popularity level of the match.  . Also, we will forward your suggestion to the concerned department',
 'We’re sorry to inform you that at present we do not have any openings. You can still drop your CV at hr@myteam11.com, so that in future if we’ll have any openings in the concerned department, we can contact you',
 'There is no issue from our side and it seems very unusual, Please share the exact error you are getting while verifying the PAN',
 'If you want to delete your existing bank account, then simply tap on the Profile Picture on the top left-hand side of our app. Go to My Profile section, Tap on . Update Details option/button and you will see your bank account number with a delete sign. Tap on that delete sign to delete your bank account details and then verify another bank account details',
 'We would like to inform you that the name on PAN card & name on Bank Account should be the same.  . As per the policies, a user must use his own documents to create and verify his account with MyTeam11 and if there is a mismatch in documents then disciplinary action may be taken',
 'Check your previous transactions in these simple steps!😊. ➡ In the MyTeam11 app, go to Wallet section. ➡ Tap on Transaction History. ➡ Click on Withdrawal to view all your previous transactions!. What would you like to do next? 👇',
 'You can check your previous transactions in these simple steps! 😊. ➡ Go to Wallet section. ➡ Tap on Transaction History. ➡ Click on Add Cash to view all your previous transactions!',
 'Sure, I can help you with that.  What would you like to know?',
 'If your account is verified, you can update or edit your information under the Info tab of the Profile section on MyTeam11.  Once the account is registered, the following information cannot be edited nor updated. - Team name. - Email. - Mobile number',
 'MyTeam11 quiz is a Q&A contest organized everyday on our mobile application. There will be a total of 8 questions in each of the quiz contest and the users will be given 8 seconds to answer each question during a round. For each correct ✔️ answer, the user will be awarded points between 1-8 based on time taken by him to answer. ',
 'This is simple, just follow the below steps 👇. ',
 'In case there’s a tie between two users for a rank in terms of right questions answered and the time taken, then the prize money for the particular rank & the immediate next rank shall be added & divided equally between the tied users. ',
 'In any such case, MyTeam11 shall not be responsible and no entry fee refunds shall be made. Further, if a user for some known/ unknown reason decides to exit the application or gets disconnected due to internet connectivity or get a call on your mobile device during a quiz round, then, the game will not pause and you will be eliminated from the game. However, the timer for answering each question of the quiz shall continue to run, and you will lose the time and will not be allowed to join back the ongoing contest. Their rank will be calculated based on total attempted questions up to that given point of time',
 'If you accidentally lock your phone while participating in a quiz contest but stay on the application, then, the game shall resume from the exact points. ',
 'We would like to inform that there will be no negative marking for any wrong answer.',
 'Any single winning of more than ₹10,000 is subjected to a TDS deduction under the Indian Income Tax Act of 1961. Thus, a tax deduction of 31.20% shall be implied on winnings more than ₹10,000',
 'There will be different types of questions asked during the quiz contest, and they shall differ for each round. However, the questions will be based on the sports industry and a bit of fantasy sports. ',
 'Hey, Daily scratch card feature is now discontinued. However, when you add an amount to your MyTeam11 account when no cash bonus offer is active you will get a scratch card. Also, we have introduced a new feature where you get a spinning wheel when you share on Facebook & Twitter that you have joined a league on our platform',
 'There is no issue from our side and it seems very unusual. However, If you are getting "time out error" then please make sure that your PAN card image is less then 2MB.  Also, If you are getting an error "PAN already verified / PAN verified in another account" that means your PAN already registered with another MyTeam11 account and we regret to inform you that you can not verify the same PAN card in two MyTeam11 accounts',
 'Please let us know about your issue in brief so that we can assist you accordingly',
 'Sure, I can help you with that',
 'Great! What would you like to know about your policy right now?',
 "Alright! I'll be around if you need my help. Please take a minute to provide feedback about this conversation! .",
 'Which of the following statements do you need right now?',
 'How would you like to make your payment?. You can pay online, through IVR, or make a cash/cheque deposit in person',
 'Perfect! Which of the following statements do you need right now?',
 'We wish to inform you that auto-debit through ECS (Electronic Clearing Service) is registered in your policy and is the best option to pay your renewal premiums due. Auto-debit of premiums through ECS frees you from the hassles of remembering the premium due dates every time as the premiums get directly debited from your bank account.',
 'Kotak Life Insurance is one of the fastest growing insurance companies in India, covering over 20 million lives nationwide.  ⭐ Quick facts —.  ✔ It is a private Life Insurance company, founded in 2001. ✔ KLI offers various protection plans, savings and investment plans, child plans and retirement plans. ',
 "That's a great decision. Kotak's Protection Plan offers you:.  ✔️ High-level protection at Nominal cost. ✔️ Savings to fulfill your dreams. ✔️ Peace of Mind. To plan a better future for you and your family, we require a few of your details.  Please tell us your name",
 'Planning for the future is a smart move. Please select exactly which category are you looking for.',
 "That's a great decision. Kotak Savings Plan offers you:.  ✔️ Dual benefit of savings & investment. ✔️ Financial Security. ✔️ More value for money. To plan a better future for you and your family, we require a few of your details.  Please tell us your name",
 "That's a great decision. Kotak's Retirement Plan offers you:.  ✔️ Financial Independence post retirement. ✔️ Wide income options suiting different retirement needs. ✔️ Sense of self-reliance & Emotional wellbeing. To plan a better future for you and your family, we require a few of your details.  Please tell us your name",
 "That's a great decision. Kotak Child Plan offers you:.  ✔️ Secure future for your children. ✔️ Power to realize their dream. ✔️ Peace of Mind. To plan a better future for you and your family, we require a few of your details.  Please tell us your name",
 'That’s Great, %name ✔️. Please help us with the name of your referral',
 "That's absolutely fine 🙂.",
 'Hi! I\'m Ally, a support chatbot run by <a href ="https://www.howtouseabortionpill.org/" target="__blank">howtouseabortionpill.org</a>. Abortion with pills (or medical abortion) is a safe and effective way to end a pregnancy up to 11 weeks. ',
 'I will try my best to share all the information I have about abortion with pills.',
 'According to research, abortion with pills is one of the safest methods and millions of people have used it with ease ✅. Abortion with pills will work successfully more than 9 out of 10 times, and the chance of a complication is less than 1% ✅. ',
 'Abortion with pills involves one or two different medicines to end a pregnancy namely Mifepristone and Misoprostol. We suggest the following -. ✅ First read through all the considerations to be kept in mind before and after using the pills so that you are fully informed & prepared.  ✅ You can also confirm if your pregnancy is under 11 weeks by using the pregnancy calculator.',
 "Here's what you need to know- .  👉 The medicine Mifepristone blocks the hormone needed for a pregnancy to grow.",
 '✅ You can eat and drink as you please during the abortion process, but avoid alcohol . ❗ You cannot eat or drink anything for 30 minutes as you wait for misoprostol to dissolve under your tongue. What would you like to know next?',
 'There are certain considerations to bear in mind before you use the pills, what can you expect after taking the pills and signs of emergency 👇. ',
 "Before using the pills, it's important to be fully informed about the abortion process. Below are common considerations and signs you should be aware of.  Choose from below to know more 👇. ",
 'Taking Ibuprofen before using the pills helps with the pain & cramping. You should also drink water through the process and eat lightly to manage the nausea ✅. Just before you use Misoprostol pills, it is best to be in a private, comfortable area where you can lie down for a few hours after you use the pills ✅. ',
 'Often women wonder if they can use the pills if they are -. ✔️ Overweight. ✔️ Pregnant with twins. ✔️ Have had a C-section before. ✔️ Have used abortion pills to terminate any past pregnancy. These do not change the safety or effectiveness of abortion pills & can be used as per the instructions.',
 "An abortion with pills is very safe, but it's important to be prepared if an emergency arises. Tap below to learn more",
 'It is important to know that abortion pills are different from methods to prevent pregnancy including IUD, emergency contraceptive pills. ',
 'If you have an Intrauterine contraceptive device in the uterus (e.g. the coil IUD or the progesterone IUD) you must get it removed before taking abortion pills. What would you like to know next?',
 'If you are breastfeeding, the misoprostol pills may cause diarrhea in the baby. To avoid this- . 👉 Breastfeed the baby first. 👉 Then take the misoprostol pills. 👉 Wait 4 hours before you breastfeed again. ',
 'If you are living with HIV, just make sure you’re not getting sicker, you’re on antiretroviral medicines, and your health is otherwise good.  You can go learn about emergency prep or read the steps to use the pills 👇',
 'Ectopic pregnancy is a condition when the pregnancy is growing outside the womb and can only be confirmed with an ultrasound. Ectopic pregnancies pose a health risk and require immediate termination. This is valid even for countries where abortion is not legal.',
 'If you have anemia (have low iron in your blood), locate a health care provider who is no more than 30 minutes away that can help if you need it.  ⚠️If you are very anemic, consult a doctor before using abortion pills',
 'You can use abortion pills in the privacy of your home or a place where you are comfortable and you can rest. We recommend using the pills sublingually (under your tongue) instead of within the vagina. As the medication dissolves completely in under 30 minutes:. ✅ There will be no evidence left of the pills. ✅ You can always state that you had a natural miscarriage.',
 'Locate the nearest 24-hour medical center that you can reach in less than 1 hour. Remember it is not safe for you to drive yourself to the center so it is better to be prepare yourself with 👇. ☑️How to get to the medical center (taxi, bus, private car) . ☑️Who can take me? (Friend, family, 24x7 transport option). ☑️The cost of travel to the center. ',
 'If the instructions are followed correctly, cramping and bleeding should start 3 hours after using Misoprostol. It is a sign that the medication is working.  ',
 'After taking Mifepristone it is normal to experience light bleeding.  3 hours after Misoprostol you may experience -. 👉 Bleeding similar to or heavier than your normal menstrual period. 👉 Pass blood clots in the first few hours.',
 'It is normal to feel feverish & nauseous after using Misoprostol. To help ease nausea-. ✅ Eat light foods. ✅ Remain extra hydrated to prevent nausea. ✅ Stay in a comfortable place & rest until you feel better. Nausea should fade away within 24 hours of using the misoprostol pills.',
 'After using Misoprostol it is normal to experience cramping. 👉Cramping commonly begins 3 hours after using Misoprostol. 👉Cramping tends to be most intense 4-8 hours after using misoprostol, though every body is different. 👉To manage cramping, ibuprofen is a good medicine to cope with pain. 💡You can take 3-4 ibuprofen 200 mg pills over the counter (without a prescription) every 6-8 hours to help with the pain. ',
 'Though bleeding and cramping are signs that the medicine is working, the following signs may require emergency medical attention or follow-up care ⚠️',
 'A pregnancy test may still be positive 2 weeks after your abortion due to lingering hormones in your body. Pregnancy symptoms should fade away within a few days of using the pills.',
 'Little or no bleeding after taking Misoprostol is a sign that the abortion did not work, or that there is a possible risk of complication. If you experienced little or no bleeding, we suggest you get in touch with trained  counselors to determine the best next steps',
 "You can use abortion pills once you have estimated your week of pregnancy. I can help you calculate this.  . Simply enter the first day of your last period in DD/MM/YY format .  💡 If my last period started on September 13, I'll enter 13/09/19 👇. . You can use abortion pills once you have estimated your week of pregnancy. I can help you calculate this.  . Simply enter the first day of your last period in DD/MM/YY format .  💡 If my last period started on September 13, I'll enter 13/09/19 👇",
 'This information varies from country to country. Please enter your country name below to learn more',
 'This information varies from country to country. Please enter your country name below to learn more',
 'This information varies from country to country. Please enter your country name below to learn more',
 'This information varies from country to country. Please enter your country name below to learn more. This information varies from country to country. Please enter your country name below to learn more',
 'This information varies from country to country. Please enter your country name below to learn more',
 'This information varies from country to country. Please enter your country name below to learn more',
 'This information varies from country to country. Please enter your country name below to learn more',
 "I'm glad you reached out. It's normal to feel a range of emotions. I can help you with some options-.  👉 If you are unsure about your pregnancy you can choose to calculate by selecting the pregnancy calculator . 👉 However, if you wish to do an abortion with pills I can help you with the details on how to use the pills. 👉 If you wish to seek further help you can chat with our counselors as well.  It is important to remember your decision is yours alone and will be private. Tap on any one option to know more 👇",
 "Thank you for reaching out. I’m Ally, a support chatbot run by howtouseabortionpill.org.  📌 HowToUse is an online community run by dedicated individuals who believe that all women, regardless of where they live, should have access to a safe abortion option. 📌 We are a global organization & we're here to provide information and resources to women around the world.  I can help you with the following information 👇",
 'Ectopic pregnancy is when a pregnancy grows outside of your uterus, usually in your fallopian tube. It can only be identified by sonography and need immediate medical assistance. What would you like to know now?',
 'Anemia is a condition in which your blood has a lower-than-normal amount of red blood cells or hemoglobin (iron-rich protein that helps red blood cells carry oxygen).  Tap on any one option to proceed 👇',
 'Miscarriage is when an embryo or fetus dies before the 20th week of pregnancy. Miscarriage usually happens early in the pregnancy.  Most miscarriages happen in the first 3 months. The medical term for miscarriage is “spontaneous abortion.”.  Tap on any one option to proceed 👇',
 'Abortion with pills has no effect on future pregnancies neither does it lead to any future birth defects. In fact, you can get pregnant again as soon as 8 days after a medical abortion.  . ❗️If you plan to have sex, you should consider using contraceptives to prevent any unplanned pregnancy. You can learn how to use the pills or reach out to our trusted counselors if you still need any assistance👇',
 'You can resume having sex as soon as you feel ready.  You also can start a new birth control method immediately after using the pills as you can get pregnant as soon as 8 days after using the pills.  You can know more about after care or choose to chat with our counselors if you are looking for further help 👇',
 'I understand that you are concerned. I can suggest some options to help you out -. 👉If you are unsure about your pregnancy you can choose to calculate by selecting the pregnancy calculator . 👉However, if you wish to do an abortion with pills I can help you with the details on how to use the pills. 👉If you wish to seek further help you can chat with our counselors as well.  Your decision will be private. Tap on any one option to know more 👇',
 'There are 2 ways to use the pills and both methods can be used safely over 24-48 hours if the instructions are followed correctly. 📌 Using pills named Mifepristone (1 pill) & Misoprostol (4 pills) in combination together.  📌Using pills named Misoprostol (8-12 pills) alone in repeated doses (Misoprostol is also commonly called Cytotec).',
 'The only way to know for sure if you are pregnant is by taking a home pregnancy test. If you have further questions you can chat with a counselor for more help',
 'We recommend using the pills sublingually (under your tongue) instead of within the vagina. As the medication dissolves completely in under 30 minutes,. ✅ There will no evidence left of the pills. ✅ You can always state that you had a natural miscarriage incase you do not wish to disclose your abortion. ',
 'There are many ways to undergo an abortion that may or may not require medical assistance.  You can use abortion pills as an option for pregnancy under 11 weeks.',
 "Alright! I'll be around if you need my help. Please take a minute to provide feedback about this conversation! .",
 "Here's what you need to know- .  👉 The medicine Mifepristone blocks the hormone needed for a pregnancy to grow.",
 'Alright. All you need to do is re-enter your full country name below to proceed 👇',
 'Hi! I\'m Ally, a support chatbot run by <a href ="https://www.howtouseabortionpill.org/" target="__blank">howtouseabortionpill.org</a>. Abortion with pills (or medical abortion) is a safe and effective way to end a pregnancy up to 11 weeks. ',
 'I’m so glad to be able to help. If you need further assistance you can continue chatting with me. Stay safe 😊',
 'I apologise that I couldn’t solve your query. If you need further assistance you can chat with our counselors as well. Stay safe 🙏',
 'An IVA can do everything that a customer service agent or sales assistant can!. I’m sure you have several questions right now! What would you like to know?',
 "Sure, I'll need your 10 digit mobile number to fetch your orders 👇.  For example: 9833XXXXXX  . Alright, I'll need your 10 digit mobile number to fetch your orders 👇.  For example: 9833XXXXXX",
 'You can get in touch on our customer care 0124-4616444 available Mon-Sat (10am -7pm) or share your query with our customer care by tapping on the button below 👇',
 "Sure, I'll need your 10 digit mobile number to fetch your order 👇.  For example: 9833XXXXXX",
 "Sure, I'll need your 10 digit mobile number to fetch your orders 👇.  For example: 9833XXXXXX",
 'We are trying hard to get the parcel to you as soon as possible. But, given the restrictions imposed, it will take around 15-18 days from the order placement date.  . Prepaid parcels will be delivered faster as courier services want to move to no exchange of cash as a precaution',
 'We are working very closely with all our vendors, courier partners and local authorities to enable deliveries as soon as possible.  In the last 2 days, some resolution has started to happen at local authority level',
 'We are now taking PAY ON DELIVERY (including CASH) orders at certain pin codes. If the option is available at your pin code, Pay on Delivery & Cash on Delivery option would be shown to you',
 "Please call our customer care at 0124-4616444 from 10 AM to 7 PM from Monday to Saturday and they will respond. If you don't receive any response, please drop a mail to care@healthkart.com",
 "We are operational but the entire head office staff is currently working from home in support of the government's directive to maintain social distancing",
 'In light of the COVID-19, some of our retail stores have been shut. Please contact the retail store before visiting. Alternatively, you can place an order on\xa0healthkart.com<b/>',
 "All the products sold on our platform are 100% authentic. As a company, we go the extra mile where we work with do rigorous Lab testing, Surprise audits and Quality checks to make sure that you never face this problem.  In case you are still not sure, we offer a 14 day no questions asked return policy. You can read about HealthKart's Legendary focus on Authentic Products & Authenticate your MB products - https://www.healthkart.com/authenticity-guaranteed",
 'Please try using prepaid payment modes or you can opt for Pay on Delivery (including CASH). We support many prepaid payment modes like debit & credit cards, EMI, Net banking, UPI, Wallets like Paytm, Amazon Pay, Freecharge, Mobikwik, Ola Money etc',
 'As a part of our stringent Quality Check policy we make sure that you never receive a product that is close to or past the expiry date. Each of our products has the expiry date printed on the Box. In case you face expiry date issues related to your product, write to us at care@healthkart.com',
 'Currently, we are shipping in India ONLY. We are listening to the feedback from customers like you to start international shipping and we may do that. Till then you can order for your loved ones in India - we deliver to more than 90% of the pin codes.  In case you want us to make an emergency shipment, write to us at care@healthkart.com We will see what best we can do for you',
 'I can help you check your product availability here. Help me with the required details.']

In [10]:
# len(questions)

232

In [11]:
# to remove Trucation warnings while running answers.append(que_generator.generate(ques))
import logging
logging.basicConfig(level=logging.ERROR)

In [15]:
# ! pip install tqdm
from tqdm import tqdm

In [16]:
answers = []
for ques in tqdm(questions):
  answers.append(que_generator.generate(ques))

100%|██████████| 232/232 [06:02<00:00,  1.56s/it]


In [19]:
result = []
a = 'The minimum versions required to support the full Cloud Edition solution is BIG-IP version 13.1.0.5 or later and BIG-IQ version 6.0.'
result.append(que_generator.generate(a))
result

[[{'answer': 'BIG-IP version 13.1.0.5 or later',
   'question': 'What is the minimum version required to support the full Cloud Edition solution?'},
  {'answer': 'BIG-IQ version 6.0',
   'question': 'What is the minimum version required to support the full Cloud Edition solution?'}]]

In [17]:
answers

[[{'answer': 'I can help you',
   'question': 'How can I help you with your order?'},
  {'answer': 'I can help you with that',
   'question': 'Can I help you with that?'},
  {'answer': '14-28 days',
   'question': 'How long will it take for your order to be delivered?'}],
 [{'answer': "Let's run through the steps",
   'question': 'How do you order a custom size?'},
  {'answer': 'custom size', 'question': 'What type of size can you order?'},
  {'answer': '2. On the page',
   'question': 'Where can you either choose from the mentioned sizes or look for the option "'},
  {'answer': 'Please click here to order a custom size',
   'question': 'What is the option to order a custom size?'},
  {'answer': '3. Enter the  ⁇ i>desired length and width ⁇ /i> in the appropriate columns',
   'question': 'How do you order a custom size?'},
  {'answer': 'columns',
   'question': 'In what area of the page do you enter the desired length and width?'},
  {'answer': "It's that simple",
   'question': 'How e